In [114]:
import pandas as pd
!pip install xlrd
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 4.7 MB/s eta 0:00:00
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [208]:
gdp_df = pd.read_csv('GDP1.csv')
gdp_df = gdp_df.fillna(-1)
### Next line unpivots table to have each year on a single row
gdp_df = pd.melt(gdp_df, id_vars='Country Name', value_vars=list(gdp_df.columns[4:]), var_name='Year', value_name='GDP')
gdp_df = gdp_df.rename(columns={'Country Name': 'Country'})
gdp_df['Year'] = gdp_df['Year'].astype('int64')

In [212]:
#contains happiness scores. From World Happiness Report
happy_data = pd.DataFrame()

for year in [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    year_df = pd.read_csv('archive-2/' + str(year) + '.csv', encoding='latin-1')
    year_df['Year'] = year
    year_df = year_df[['Country', 'Happiness Rank', 'Happiness Score',
       'Year']]
    year_df = year_df.dropna()
    happy_data = pd.concat([happy_data, year_df])

In [213]:
# contains all other info about how happiness score was derived. From World Happiness Report
happy_data1 = pd.read_excel('DataForTable2.1.xls')
happy_data1 = happy_data1.rename(columns={'Country name': 'Country', 'year': 'Year'})

In [214]:
happy_df = pd.merge(happy_data1, happy_data, on=['Country', 'Year'], how='left')#left_on=["Country name", 'year'], right_on=['Country', 'Year'], how='left')

In [219]:
happy_df = happy_df[happy_df['Year'] >= 2012]

In [220]:
le_df = pd.read_csv('Life_expectancy/Life_Expectancy.csv')
le_df = le_df.fillna(-1)
### Next line unpivots table to have each year on a single row
le_df = pd.melt(le_df, id_vars='Country Name', value_vars=list(le_df.columns[4:]), var_name='Year', value_name='Life Expectancy')
le_df = le_df.rename(columns={'Country Name': 'Country'})
le_df['Year'] = le_df['Year'].astype('int64')

In [221]:
xlsx = pd.ExcelFile('Freedom.xlsx')
freedom_df = pd.read_excel(xlsx, 'FIW13-22', header=1)
freedom_df = freedom_df.rename(columns={'Country/Territory': 'Country', 'Edition': 'Year', 'Total': 'Freedom_Rating'})
freedom_df = freedom_df[['Country', 'Year', 'Freedom_Rating']]

In [222]:
# merge all dfs
df = happy_df.merge(freedom_df, on=['Country', 'Year'], how='left')
df = df.merge(le_df, on=['Country', 'Year'], how='left')
df = df.merge(gdp_df, on=['Country', 'Year'], how='left')

In [206]:
df.to_csv('all_data.csv')

In [223]:
df

,Country,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Happiness Rank,Happiness Score,Freedom_Rating,Life Expectancy,GDP
0,Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,0.613513,0.267919,0.435440,NaN,NaN,NaN,62.054,1.990732e+10
1,Afghanistan,2013,3.572100,7.657503,0.483552,52.000000,0.577955,0.066344,0.823204,0.547417,0.273328,0.482847,NaN,NaN,27.0,62.525,2.014640e+10
2,Afghanistan,2014,3.130896,7.650827,0.525568,52.299999,0.508514,0.109215,0.871242,0.491641,0.374861,0.409048,NaN,NaN,26.0,62.966,2.049713e+10
3,Afghanistan,2015,3.982855,7.634466,0.528597,52.599998,0.388928,0.085082,0.880638,0.491410,0.339276,0.260557,153.0,3.575,24.0,63.377,1.913421e+10
4,Afghanistan,2016,4.220169,7.629037,0.559072,52.924999,0.522566,0.047488,0.793246,0.501409,0.348332,0.324990,154.0,3.36,24.0,63.763,1.811656e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,Zimbabwe,2017,3.638300,8.241609,0.754147,52.150002,0.752826,-0.113937,0.751208,0.733641,0.224051,0.682647,138.0,3.875,32.0,60.812,1.758489e+10
1373,Zimbabwe,2018,3.616480,8.274620,0.775388,52.625000,0.762675,-0.084747,0.844209,0.657524,0.211726,0.550508,144.0,3.692,30.0,61.195,1.811554e+10
1374,Zimbabwe,2019,2.693523,8.196998,0.759162,53.099998,0.631908,-0.081540,0.830652,0.658434,0.235354,0.456455,146.0,3.663,31.0,61.490,1.928429e+10
1375,Zimbabwe,2020,3.159802,8.117733,0.717243,53.575001,0.643303,-0.029376,0.788523,0.660658,0.345736,0.577302,151.0,3.2992,29.0,61.738,1.805117e+10
